<a href="https://colab.research.google.com/github/660sun/Computational-Photonics-code-3/blob/main/Homework3_report_group5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

<p style="font-size:30px;"><strong> Computational Photonics </p>

<p style="font-size:30px;"><strong> Homework 3: Implementation of the 1D and 3D version of the FDTD method </p>


</center>

<center>

**Author:**
*Group 5*

| Name             | Email       |
| -----------      | ----------- |
| *Lena Fleischmann*   |  *l.fleischmann@uni-jena.de*   |
| *Felix Kreter* | *felix.kreter@uni-jena.de*
| *Yucheng Sun*     |  *yucheng.sun@uni-jena.de*        |
| *Nayana Jalimarad Shankarappa* | *nayana.jalimarad.shankarappa@uni-jena.de*|

</center>

>**Supervisor:**
>
>*Prof. Thomas Pertsch*
>
>**Tutors:**
>
>*Tobias Bucher*
>
>*Jan Sperrhake*

In [ ]:
import numpy as np
import time
from matplotlib import pyplot as plt

# from function_headers_fdtd import Fdtd1DAnimation, Fdtd3DAnimation

# dark bluered colormap, registers automatically with matplotlib on import
import bluered_dark


plt.rcParams.update({
        'figure.figsize': (12/2.54, 9/2.54),
        'figure.subplot.bottom': 0.15,
        'figure.subplot.left': 0.165,
        'figure.subplot.right': 0.90,
        'figure.subplot.top': 0.9,
        'axes.grid': False,
        'image.cmap': 'bluered_dark',
})

plt.close('all')

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

**Table of contents**<a id='toc0_'></a>    
- 1. [Introduction](#toc1_)    
- 2. [Finite-Difference Time-Domain (FDTD) Method](#toc2_)    
  - 2.1. [Maxwell's equations](#toc2_1_)    
  - 2.2. [Yee grids and indices](#toc2_2_)    
    - 2.2.1. [Discretization](#toc2_2_1_)    
    - 2.2.2. [Resolution consideration](#toc2_2_2_)    
    - 2.2.3. [Yee grid in 1+1D](#toc2_2_3_)    
    - 2.2.4. [4D Yee grid](#toc2_2_4_)    
  - 2.3. [Perfect electric conductor boundary](#toc2_3_)    
- 3. [Analysis and Simulation of the Problems](#toc3_)    
  - 3.1. [Task 1 - 1D FDTD](#toc3_1_)    
    - 3.1.1. [Implementation](#toc3_1_1_)    
    - 3.1.2. [Convergence test](#toc3_1_2_)    
    - 3.1.3. [Example: 1D](#toc3_1_3_)    
  - 3.2. [Task 2 - 3D FDTD](#toc3_2_)    
    - 3.2.1. [Implementation](#toc3_2_1_)    
    - 3.2.2. [Example](#toc3_2_2_)    
- 4. [Conclusion](#toc4_)    
- 5. [References](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Introduction](#toc0_)

In this project, we employ the finite-difference time-domain (FDTD) method to simulate the propagation of an ultrashort pulse in a dispersion-free dielectric medium in both 1D and 3D cases. And we use *Python* to implement this method. In addition, both the physical properties and the numerical properties (i.e. the convergence test of the discretization in geometry discretization and also in time) of the simulation will be discussed. Specifically, the phenomenon when the pulse hits the interface between two different dielectric media will also be investigated.

## 2. <a id='toc2_'></a>[Finite-Difference Time-Domain (FDTD) Method](#toc0_)

The Finite-Difference Time-Domain (FDTD) Method solves the Maxwell's equations in time domain. Here the assumption of a non-dispersive material is used (which is unphysical). From this we can write:
$$
\mathbf{D}(\mathbf{r},t)=\varepsilon_0\varepsilon(\mathbf{r})\mathbf{E}(\mathbf{r},t)\tag{1-1}
$$
which leads the curl equation of the Maxwell's equations for the $\mathbf{H}$-field as given in 2.1.

### 2.1. <a id='toc2_1_'></a>[Maxwell's equations](#toc0_)

\begin{align}
\frac{\partial \textbf{H}(\textbf{r}, t)}{\partial t} &= - \frac{1}{\mu_0} \nabla \times \textbf{E}(\textbf{r}, t) \tag{2-1}\\
&\downarrow \notag \\
\frac{\partial H_x}{\partial t} &= \frac{1}{\mu_0}\Big[ \frac{\partial E_y}{\partial z} - \frac{\partial E_z}{\partial y} \Big], \tag{2-2}\\
\frac{\partial H_y}{\partial t} &= \frac{1}{\mu_0}\Big[ \frac{\partial E_z}{\partial x} - \frac{\partial E_x}{\partial z} \Big], \tag{2-3}\\
\frac{\partial H_z}{\partial t} &= \frac{1}{\mu_0}\Big[ \frac{\partial E_x}{\partial y} - \frac{\partial E_y}{\partial x} \Big] \tag{2-4}
\end{align}

\begin{align}
\frac{\partial \textbf{E}(\textbf{r}, t)}{\partial t} &= \frac{1}{\epsilon_0\epsilon(\textbf{r})} \Big[ \nabla \times \textbf{H}(\textbf{r}, t) - \textbf{j}(\textbf{r}, t) \Big] \tag{2-5}\\
&\downarrow \notag \\
\frac{\partial E_x}{\partial t} &= \frac{1}{\epsilon_0\epsilon(\textbf{r})}\Big[ \frac{\partial H_z}{\partial y} - \frac{\partial H_y}{\partial z} - j_x \Big], \tag{2-6}\\
\frac{\partial E_y}{\partial t} &= \frac{1}{\epsilon_0\epsilon(\textbf{r})}\Big[ \frac{\partial H_x}{\partial z} - \frac{\partial H_z}{\partial x} - j_y \Big], \tag{2-7}\\
\frac{\partial E_z}{\partial t} &= \frac{1}{\epsilon_0\epsilon(\textbf{r})}\Big[ \frac{\partial H_y}{\partial x} - \frac{\partial H_x}{\partial y} -j_z \Big] \tag{2-8}
\end{align}

### 2.2. <a id='toc2_2_'></a>[Yee grids and indices](#toc0_)

#### 2.2.1. <a id='toc2_2_1_'></a>[Discretization](#toc0_)

In order to implement the equations given in 2.1 numerically, the space $(x,y,z)$ and time $(t)$ is dicretized using the indices $(i,j,k)$ and $(n)$ as follows:
$$
E_x(x,y,z,t) \rightarrow E_x(i\Delta x, j\Delta y, k \Delta z, n \Delta t) = E_x|_{ijk}^n\tag{2-9}
$$
and
$$
\varepsilon(x,y,z) \rightarrow \varepsilon(i\Delta x, j\Delta y, k \Delta z)=\varepsilon_{ijk}\tag{2-10}
$$

Furthermore, using the central finite difference, the derivative operator in space and time is also discretized:
$$
\frac{\partial E_x|_{ijk}^n}{\partial y} \approx \frac{E_x|_{ij+1k}^n - E_x|_{ij-1k}^n}{2\Delta y}\tag{2-11}
$$
$$
\frac{\partial E_x|_{ijk}^n}{\partial t} \approx \frac{E_x|_{ijk}^{n+1} - E_x|_{ijk}^{n-1}}{2\Delta t}\tag{2-12}
$$

And the magnetic field should be treated in the same way.

#### 2.2.2. <a id='toc2_2_2_'></a>[Resolution consideration](#toc0_)

Another point to introduce are the resolution limits of the discretization sizes.
The spatial grid resolution $\Delta x$ must be fine enough to represent the samllest structures of the $\varepsilon$ distribution and the fields. For this usually the condition $\Delta x = \lambda/(20n_{max})$ is introduced. Whereas $n_{max}$ describes the maximal value of the refractive index in the simulated area.
Furthermore the temporal step size $\Delta t$ is limited due to speed of light $c$ with the condition $\Delta t \leq \Delta x/c$, which is called the spatial-temporal discretization scheme.

#### 2.2.3. <a id='toc2_2_3_'></a>[Yee grid in 1+1D](#toc0_)






For the one-dimensional case, it is assumed that there is no change of the fields in $y$ and $z$ direction. For this case, these equations reduce by the dependence on $y$ and $z$ as well as the corresponding indices $j$ and $k$.

The equations for $\textbf{E}$ and $\textbf{H}$ in 1D are then given by (here the z- and y-component, respectively, because of assumed invariance of the fields in z and y):
$$
E_z|_i^{n+1} \approx E_z|_i^{n}+\frac{1}{\varepsilon_0\varepsilon_i}\frac{\Delta t}{\Delta x}[H_y|_{i+0.5}^{n+0.5} - H_y|_{i-0.5}^{n+0.5}]-\frac{\Delta t}{\varepsilon_0\varepsilon_i}j_z|_{i}^{n+0.5}\tag{2-13}
$$
where $j_z$ is the source term (see the implementation part in section 3), and
$$
H_y|_{i+0.5}^{n+1.5} \approx H_y|_{i+0.5}^{n+0.5}+\frac{1}{\mu_0}\frac{\Delta t}{\Delta x}[E_z|_{i+1}^{n+1} - E_z|_{i}^{n+1}]\tag{2-14}
$$

Due to the central difference derivative operator in space and time the following holds: For the the calculation of the $\textbf{E}$-field at a certain position in space and time, we don't need the $\textbf{E}$-field of the time step before but of the one two before and we don't need the $\textbf{H}$-field at the same spatial and temporal position as the $\textbf{E}$-field but one at time step before and at one step before and after in space. The same holds for the $\textbf{H}$-field accordingly.

We do not need the $\textbf{E}$-field and the $\textbf{H}$-field at the same spatial and temporal position which makes sence regarding Ampere's law and Faraday's law. To save memory and simplify the grid, one can use the discretization introduced by Yee, called "Yee grid". In this we have the grids of $\textbf{E}$ and $\textbf{H}$ shifted by half an index step in space and time.

Since for the implementation half indices are not feasible, all half indices above are relabeled: $n+1/2 \rightarrow n$. Then we can have

$$
E_z|_i^{n+1} \approx E_z|_i^{n}+\frac{1}{\varepsilon_0\varepsilon_i}\frac{\Delta t}{\Delta x}[H_y|_{i}^{n} - H_y|_{i}^{n}]-\frac{\Delta t}{\varepsilon_0\varepsilon_i}j_z|_{i}^{n+0.5}\tag{2-15}
$$
where $j_z$ is the source term (see the implementation part in section 3), and
$$
H_y|_{i}^{n+1} \approx H_y|_{i}^{n}+\frac{1}{\mu_0}\frac{\Delta t}{\Delta x}[E_z|_{i+1}^{n+1} - E_z|_{i}^{n+1}]\tag{2-16}
$$
Here, we keep the temproal index of the source term as before to highlight the time of the source when processing it numerically, which will have a great influence on the final results.

The future field (one field component of $\textbf{E}$ and $\textbf{H}$ each in 1D) is calculated by the fields of the other kind ($\textbf{H}$ or $\textbf{E}$) in the present but spatially one step to the right and left and the same field ($\textbf{E}$ or $\textbf{H}$) in the past, respectively. For the next time step the other field ($\textbf{H}$ or $\textbf{E}$) is calculated with the fields which were just calculated and again the same field two time steps before.

To start we need to know the electric field at the index $n=0$ and the magnetic field at the index $n=0.5$, so at a little later time step. In this simulation these fields are set to zero and we add a source (see implementation part for details).

#### 2.2.4. <a id='toc2_2_4_'></a>[4D Yee grid](#toc0_)


The 4D Yee grid (3+1D) consists of the three spatial dimensions and the temporal dimension. The field components of $\textbf{E}$ and $\textbf{H}$ are in a distance of half spatial step and half a time step in the grid. The illustration of the grid, which has a cubic structure, only shows - unlike for the 1D case - the spatial distribution for $\textbf{E}$ at a certain time and at half a time step later for $\textbf{H}$.

For the permittivity which is used in the calculation of the $\textbf{E}$-field an interpolation to half the step size forward in all three dimensions needs to be done (here for the x-direction, for y and z accordingly):

$$
\frac{1}{\varepsilon_{i+0.5,j,k}}=\frac{1}{2}\biggl( \frac{1}{\varepsilon_{i,j,k}}+\frac{1}{\varepsilon_{i+1,j,k}} \biggr)
$$

Again the indices are renamed in this way:
$$
i + 0.5 \rightarrow i
$$
(for $j$ and $k$ accordingly).
The interpolated permittivity for $x$ then becomes:
$$
\varepsilon_{i+0.5,j,k} \rightarrow \varepsilon_x|_{i,j,k}
$$
(for $y$ and $z$ with $j$ and $k$ accordingly). In the 3D case, since we assume that the material is spatially independent. For simplicity, it is enough to discretized the permittivity without interpolation.

The equations for the calculation of the $\textbf{E}$ and $\textbf{H}$ fields are given by (already with the renamed indices):

$$
E_x|_{i,j,k}^{n+1} = E_x|_{i,j,k}^{n} + \frac{\Delta t}{\varepsilon_0 \varepsilon_x|_{i,j,k}} \biggl ( \frac{H_z|_{i,j,k}^{n} - H_z|_{i,j-1,k}^{n}}{\Delta y} - \frac{H_y|_{i,j,k}^{n} - H_y|_{i,j,k-1}^{n}}{\Delta z} - j_x|_{i,j,k}^n \biggr ),\tag{2-17}
$$

$$
E_y|_{i,j,k}^{n+1} = E_y|_{i,j,k}^{n} + \frac{\Delta t}{\varepsilon_0 \varepsilon_y|_{i,j,k}} \biggl ( \frac{H_x|_{i,j,k}^{n} - H_x|_{i,j,k-1}^{n}}{\Delta z} - \frac{H_z|_{i,j,k}^{n} - H_z|_{i-1,j,k}^{n}}{\Delta x} - j_y|_{i,j,k}^n \biggr ),\tag{2-18}
$$

$$
E_z|_{i,j,k}^{n+1} = E_z|_{i,j,k}^{n} + \frac{\Delta t}{\varepsilon_0 \varepsilon_z|_{i,j,k}} \biggl ( \frac{H_y|_{i,j,k}^{n} - H_y|_{i-1,j,k}^{n}}{\Delta x} - \frac{H_x|_{i,j,k}^{n} - H_y|_{i,j-1,k}^{n}}{\Delta y} - j_z|_{i,j,k}^n \biggr ),\tag{2-19}
$$

$$
H_x|_{i,j,k}^{n+1} = H_x|_{i,j,k}^{n} + \frac{\Delta t}{\mu_0 } \biggl ( \frac{E_y|_{i,j,k+1}^{n+1} - E_y|_{i,j,k}^{n+1}}{\Delta z} - \frac{E_z|_{i,j+1,k}^{n+1} - E_z|_{i,j,k}^{n+1}}{\Delta y} \biggr ),\tag{2-20}
$$

$$
H_y|_{i,j,k}^{n+1} = H_y|_{i,j,k}^{n} + \frac{\Delta t}{\mu_0 } \biggl ( \frac{E_z|_{i+1,j,k}^{n+1} - E_z|_{i,j,k}^{n+1}}{\Delta x} - \frac{E_x|_{i,j,k+1}^{n+1} - E_x|_{i,j,k}^{n+1}}{\Delta z} \biggr ),\tag{2-21}
$$

$$
H_z|_{i,j,k}^{n+1} = H_z|_{i,j,k}^{n} + \frac{\Delta t}{\mu_0 } \biggl ( \frac{E_x|_{i,j+1,k}^{n+1} - E_x|_{i,j,k}^{n+1}}{\Delta y} - \frac{E_y|_{i+1,j,k}^{n+1} - E_y|_{i,j,k}^{n+1}}{\Delta x} \biggr ).\tag{2-22}
$$

### 2.3. <a id='toc2_3_'></a>[Perfect electric conductor boundary](#toc0_)

According to mathematics, to solve the Maxwell equations, we also need to know the boundary conditions for a finite computational domain. Here we use perfect electric conductor boundary, which states that all the fields at the outside of the boundaries are set to be $0$.

The boundary conditions in electromagnetic also states that the tangential components of the electric fields ($\textbf{E}$) and the normal components of the magnetic flux density ($\textbf{B}$) should continue at the boundaries. This means these components should also be zero at the inside of the boundaries. In the following, it will be shown more detailedly.

1. For the $E_x$, it should be zero at plane $y=0$, $y=y_{max}$, $z=0$ and $z=z_{max}$.

2. For the $E_y$, it should be zero at plane $x=0$, $x=x_{max}$, $z=0$ and $z=z_{max}$.

3. For the $E_z$, it should be zero at plane $x=0$, $x=x_{max}$, $y=0$ and $y=y_{max}$.

4. For the $H_x$, it should be zero at plane $x=0$ and $x=x_{max}$.

5. For the $H_y$, it should be zero at plane $y=0$ and $y=y_{max}$.

6. For the $H_z$, it should be zero at plane $z=0$ and $z=z_{max}$.

<center>
    
<img src="Computational_Domain.svg" alt="Computational_Domain" width="500" height="400"/>

Figure 2-1: Computational domain.

</center>

Numerically, we need to set the following values to zero:
$E_x(:, 1, :)$,
$E_x(:, N_y, :)$,
$E_x(:, :, 1)$,
$E_x(:, :, N_z)$,
$E_y(1, :, :)$,
$E_y(N_x, , :)$,
$E_y(:, :, 1)$,
$E_y(:, :, N_z)$,
$E_z(1, :, :)$,
$E_z(N_x, , :)$,
$E_z(:, 1, :)$,
$E_z(:, N_y, :)$,
$H_x(1, :, :)$,
$H_x(N_x, , :)$,
$H_y(:, 1, :)$,
$H_y(:, N_y, :)$,
$H_z(:, :, 1)$,
$H_z(:, :, N_z)$.

Here we use the general index which starts from $1$ while operating these in *Python* we need to substract all the indices of them by 1 since the index in *Python* will always begin with $0$.



## 3. <a id='toc3_'></a>[Analysis and Simulation of the Problems](#toc0_)

In this part, we will show how we implement the FDTD method numerically using *Python*.

### 3.1. <a id='toc3_1_'></a>[Task 1 - 1D FDTD](#toc0_)

- Basic parameters - 1D

In [ ]:
# constants
c = 2.99792458e8 # speed of light [m/s]
mu0 = 4*np.pi*1e-7 # vacuum permeability [Vs/(Am)]
eps0 = 1/(mu0*c**2) # vacuum permittivity [As/(Vm)]
Z0 = np.sqrt(mu0/eps0) # vacuum impedance [Ohm]

# geometry parameters
x_span = 18e-6 # width of computatinal domain [m]
n1 = 1 # refractive index in front of interface
n2 = 2 # refractive index behind interface
x_interface = x_span/4 #postion of dielectric interface

# source parameters
source_frequency = 500e12 # [Hz]
source_position = 0 # [m]
source_pulse_length = 1e-15 # [s]

# temporal parameter
time_span = 60e-15 # duration of simulation [s]


This part defines the basic parameters of the simulation problem, including some constants, geometrical structure and refractive index of the computation domain, and the important parameters for the source.

The source we used here is a Gaussian-modulated sinusoidal pulse introduced at a specified position, having the distribution in space and time given as follows:
\begin{align}
j_z(t, x)=j_0\delta(x-x_0)\times \exp\Big[-\Big(\frac{t-t_0}{\tau}\Big)^2\Big]\times \exp(-i*2\pi ft).\tag{3-1}
\end{align}
Here we use the sign $\times$ to distinguish different parts of the source term.

The first part indicate that the source is point source which is placed at the specific position $x_0$, i.e., the *source_position* in our parameters. The second term indicates that it has a Gaussian envelope while the third term states that the source oscillate with a frequency $f$. In the codes, we choose to use $\cos(2\pi ft)$ to insted the complex representation.

We can know from the source term that the field we will obtain in 1D must also be a pulse. Besides, the oscillating frequency also tells us that the discretized size in spatial domain should be smaller than
\begin{align}
dx < c_0 \times \frac{1}{f} = 60nm.
\end{align}

#### 3.1.1. <a id='toc3_1_1_'></a>[Implementation](#toc0_)

In [ ]:
def fdtd_1d(eps_rel, dx, time_span, source_frequency, source_position, source_pulse_length):
    '''Computes the temporal evolution of a pulsed excitation using the
    1D FDTD method. The temporal center of the pulse is placed at a
    simulation time of 3*source_pulse_length. The origin x=0 is in the
    center of the computational domain. All quantities have to be
    specified in SI units.

    Arguments
    ---------
        eps_rel : 1d-array
            Rel. permittivity distribution within the computational domain.
        dx : float
            Spacing of the simulation grid (please ensure dx <= lambda/20).
        time_span : float
            Time span of simulation.
        source_frequency : float
            Frequency of current source.
        source_position : float
            Spatial position of current source.
        source_pulse_length :
            Temporal width of Gaussian envelope of the source.

    Returns
    -------
        Ez : 2d-array
            Z-component of E(x,t) (each row corresponds to one time step)
        Hy : 2d-array
            Y-component of H(x,t) (each row corresponds to one time step)
        x  : 1d-array
            Spatial coordinates of the field output
        t  : 1d-array
            Time of the field output
    '''

    # basic parameters
    c = 2.99792458e8 # speed of light [m/s]
    mu0 = 4*np.pi*1e-7 # vacuum permeability [Vs/(Am)]
    eps0 = 1/(mu0*c**2) # vacuum permittivity [As/(Vm)]
    Z0 = np.sqrt(mu0/eps0) # vacuum impedance [Ohm]

    # time step
    dt = dx / (2 * c)
    Nt = int(round(time_span / dt)) + 1
    t = np.linspace(0, time_span, Nt)

    # construction of matrices
    Ez = np.zeros((Nt, len(eps_rel)))
    Hy = np.zeros((Nt, len(eps_rel)))
    jz = np.zeros((Nt, len(eps_rel) - 1))

    # spatial coordinates of the fields
    x = np.linspace(-(len(eps_rel) - 1)/2*dx, (len(eps_rel) - 1)/2*dx, len(eps_rel))

    # source matrix
    for n in range(Nt):
        jz[n, source_position] = np.exp(-(((n + 0.5)*dt - 3*source_pulse_length)/source_pulse_length)**2) * np.cos(2*np.pi*source_frequency * (n + 0.5)*dt)

    # main loop
    for n in range(1, Nt):
        for i in range(1, len(eps_rel) - 1):
            Ez[n, i] = Ez[n-1, i] + (Hy[n-1, i] - Hy[n-1, i-1]) * 1 / ( eps0 * eps_rel[i] ) * dt / dx - jz[n-1, i] * dt / ( eps0 * eps_rel[i] )
        for i in range(len(eps_rel) - 1):
            Hy[n, i] = Hy[n-1, i] + (Ez[n, i+1] - Ez[n, i]) * 1 / mu0 * dt / dx

    # postprocessing - interpolation of output
    for n in range(1, len(Ez)):
        Hy[n, 0] = 0.5 * (Hy[n, 0] + Hy[n-1, 0])
        Hy[n, -1] = 0.5 * (Hy[n, -2] + Hy[n-1, -2])
        for i in range(1, len(eps_rel)-1):
            Hy[n, i] = 0.25 * (Hy[n, i] + Hy[n, i-1] + Hy[n-1, i] + Hy[n-1, i-1])

    return Ez, Hy, x, t

The provided code implements a 1D FDTD simulation to compute the temporal evolution of an electromagnetic wave excited by a pulsed source. It takes the relative permittivity distribution (*eps_rel*), grid spacing (*dx*), total simulation time (*time_span*), source frequency (*source_frequency*), source position (*source_position*), and pulse length (*source_pulse_length*) as inputs. The simulation grid is defined by *dx* and spans the given permittivity distribution, ensuring that spatial steps are sufficiently small relative to the wavelength of the source.

Since the problem should be solved in time domain, we need to give an appropriate temproal discretization parameter *dt*. According to Courant condition, we need to set the *dt* to be smaller than $dx/(\sqrt{n}c)$ to convergent our program, where $n$ is the dimension of the simulation problem. Here we set *dt* to be $dx/(2c)$. (line*41-44*)

The code initializes the electric field $Ez$ and magnetic field $Hy$ matrices, which are updated iteratively at each time step according to Maxwell's equations. And the source matrix is also initailized. (line *46-49*)

The main loop computes the electric and magnetic fields at each grid point and time step by updating $Ez$ and $Hy$ based on their previous values and the source term. Here we implement the main loop using the *for* loop. Since this is 1D case, the *for* loop will not take too much time but show a more clear concept of the FDTD method. (line *58-63*)

Postprocessing interpolates the magnetic field values to ensure a smooth transition between time steps, reflecting the proper boundary conditions. (line *65-70*) The function returns the electric and magnetic field distributions over space and time, along with the spatial (*x*) and temporal (*t*) coordinates.

#### 3.1.2. <a id='toc3_1_2_'></a>[Convergence test](#toc0_)

The implemented algorithm depends on two parameters, that determine its accuracy, i.e. the discretized size in the x direction, which is represented by $dx$ in the problem, and the discretized size in the time domain, which is represented by $dt$ in the problem. These parameters determine the accuracy of the geometrical and temporal representation and the accuracy of the finite difference approximation of the differential operator. So we implement the convergence tests for both $dx$ and $dt$. And these are the same for the 3D case.

**1. Convergence test for dx**

In the following the convergence of the discretization sizes are tested. In this case the convergence for the spatial discretization $dx$ is varied. For each value of $dx$ the error based on a reference is calculated as well as the computational time.

In the first step the reference values are calculated as *Ez_0*, *Hy_0*, *x_0* and *t_0* with the defined function for the 1D case. For the reference fields the initial parameter are used. For a change of the discretization steps also teh number of grid points is either reduced or increased so that the resulting fields need to be interpolated. The initial values or parameters are more preferred than using for example smallest discretization steps, resulting in higher precision. Smaller discretization steps also result in higher computational effort.

In [ ]:
# simulation parameters
dx = 15e-9 # grid spacing [m]
time_span = 60e-15 # duration of simulation [s]

Nx = int(round(x_span/dx)) + 1 # number of grid points

# create reference calculation with initial parameters

x = np.linspace(-x_span/2, x_span/2, Nx)

eps_rel = np.ones(Nx)
for i in range(Nx):
    if x[i] > x_interface:
        eps_rel[i] = n2**2
    else:
        eps_rel[i] = n1**2

for i, xi in enumerate(x):
    if abs(xi - source_position) < dx/2:
        source_position = i

# calculate reference field with initial value for dx
Ez_0, Hy_0, x_0, t_0 = fdtd_1d(eps_rel, dx, time_span, source_frequency, source_position, source_pulse_length)

Now the convergence for $dx$ is observed by obtaining the $L_2$  norm error and the operation time for different values for $dx$. Due to the coupling of $dx$ and $dt$ a change of the spatial discretization will result also in a change of temporal discretization.

In [ ]:
# convergence for dx

# dx = np.logspace(-8, -7, num=10)
dx = np.linspace(10*1e-9,20*1e-9,10)
# print(dx)

errors = np.zeros(len(dx))
operation_time = np.zeros(len(dx))

for i, dxi in enumerate(dx):
    start = time.time()
    Nx = int(round(x_span/dxi)) + 1
    x = np.linspace(-x_span/2, x_span/2, Nx)
    # permettivity with new step size
    eps_rel = np.ones(Nx)
    for j in range(Nx):
        if x[j] > x_interface:
            eps_rel[j] = n2**2
        else:
            eps_rel[j] = n1**2

    for l, xi in enumerate(x):
        if abs(xi - source_position) < dxi/2:
            source_position = l

    Ez, Hy, x, t= fdtd_1d(eps_rel, dxi, time_span, source_frequency, source_position, source_pulse_length)
    end = time.time()
    operation_time[i] = end - start


    # Interpolate Ez to match the reference grid
    Ez_interp = np.interp(x_0, x, Ez[-1])
    Ez_ref_interp = Ez_0 #np.interp(x, x_0, Ez_0[-1]) #reference grid do not need to be interpolated since it's fixed

    # Compute L2 norm of the error
    errors[i] = np.linalg.norm(Ez_interp - Ez_ref_interp) / np.linalg.norm(Ez_ref_interp)
    print(f"dt = {dxi:.2e}, runtime = {end-start:.2f} s, L2 error = {errors[i]:.2e}")

To visualize the results in the following both, the operation time and the error defined by the norm are plotted.

In [ ]:
# Plot operation time vs. time step
plt.figure()
plt.plot(dx*1e9, operation_time, 'o-')
plt.xlabel('spatial step $dx$ [nm]')
plt.ylabel('Operation time [s]')
# plt.xscale('log')  # Use logarithmic scale for better visualization
# plt.yscale('log')
plt.grid()
plt.title('Operation Time vs. Spatial Step')
plt.savefig('comp_time_dx.png', bbox_inches="tight")
plt.show()

# Plot L2 error norm vs. time step
plt.figure()
plt.plot(dx*1e9, errors, 'o-')
plt.xlabel('spatial step $dx$ [nm]')
plt.ylabel('L2 Norm of Error')
# plt.yscale('log')
plt.grid()
plt.title('L2 Norm of Error vs. Spatial Step')
plt.savefig('error_dx.png', bbox_inches="tight")
plt.show()

<center>
    
<img src="1d_convergence_dx_time.svg" alt="1d_convergence_dx_time" width="800" height="400"/>

Figure 3-1: The operation time for different discretized size of x direction in 1D FDTD.

</center>

<center>
    
<img src="1d_convergence_dx_error.svg" alt="fdtd_1d_ez_postprocessing_snapshot" width="800" height="400"/>

Figure 3-2: $L_2$ norm errors for different discretized size of x direction in 1D FDTD.

</center>

When having a look at the convergence test for the spatial step size $dx$ it is visible that an increase of the step size would result in less precision during the calculations which therefore result in an increase of the error. Next to this a larger discretization step size decrease the number of grid points in the computational area the operation time is decreased. To choose a good value one should keep in mind the computational effort and the precision of the grid which is needed.

**2. Convergence test for dt**

Important to note is that a change of $dt$ is just valid in a certain range, when keeping $dx$ as a constant. Without respecting the condition $\Delta t \leq \Delta x/c$ a variation of the time discretization step would result in energy violation.

To show this a new function for the 1D case is introduced in which $dt$ and the additional number of time points $N_t$ are also variables.

In [ ]:
def fdtd_1d_t(eps_rel, dx, dt, Nt, source_frequency, source_position, source_pulse_length):

    # basic parameters
    c = 2.99792458e8 # speed of light [m/s]
    mu0 = 4*np.pi*1e-7 # vacuum permeability [Vs/(Am)]
    eps0 = 1/(mu0*c**2) # vacuum permittivity [As/(Vm)]

    # construction of matrices
    Ez = np.zeros((Nt, len(eps_rel)))
    Hy = np.zeros((Nt, len(eps_rel)))
    jz = np.zeros((Nt, len(eps_rel) - 1))

    # spatial coordinates of the fields
    x = np.linspace(-(len(eps_rel) - 1)/2*dx, (len(eps_rel) - 1)/2*dx, len(eps_rel))

    # source matrix
    for n in range(Nt):
        jz[n, source_position] = np.exp(-(((n + 0.5)*dt - 3*source_pulse_length)/source_pulse_length)**2) * np.cos(2*np.pi*source_frequency * (n + 0.5)*dt)

    # main loop
    for n in range(1, Nt):
        for i in range(1, len(eps_rel) - 1):
            Ez[n, i] = Ez[n-1, i] + (Hy[n-1, i] - Hy[n-1, i-1]) * 1 / ( eps0 * eps_rel[i] ) * dt / dx - jz[n-1, i] * dt / ( eps0 * eps_rel[i] )
        for i in range(len(eps_rel) - 1):
            Hy[n, i] = Hy[n-1, i] + (Ez[n, i+1] - Ez[n, i]) * 1 / mu0 * dt / dx

    # postprocessing - interpolation of output
    for n in range(1, len(Ez)):
        Hy[n, 0] = 0.5 * (Hy[n, 0] + Hy[n-1, 0])
        Hy[n, -1] = 0.5 * (Hy[n, -2] + Hy[n-1, -2])
        for i in range(1, len(eps_rel)-1):
            Hy[n, i] = 0.25 * (Hy[n, i] + Hy[n, i-1] + Hy[n-1, i] + Hy[n-1, i-1])

    return Ez, Hy, x

In [ ]:
# simulation parameters
dx = 15e-9 # grid spacing [m]
time_span = 60e-15 # duration of simulation [s]

Nx = int(round(x_span/dx)) + 1 # number of grid points

# create reference calcualtion with initial parameters and initial function for the 1D case

x = np.linspace(-x_span/2, x_span/2, Nx)

eps_rel = np.ones(Nx)
for i in range(Nx):
    if x[i] > x_interface:
        eps_rel[i] = n2**2
    else:
        eps_rel[i] = n1**2

for i, xi in enumerate(x):
    if abs(xi - source_position) < dx/2:
        source_position = i

Ez_0, Hy_0, x_0, t_0 = fdtd_1d(eps_rel, dx, time_span, source_frequency, source_position, source_pulse_length)

Now the convergence for $dt$ is observed by obtaining the $L_2$  norm error and the operation time for different values for $dt$.

In [ ]:
dt = np.linspace(1e-17,1e-14,10)
# print(dt)

errors = np.zeros(len(dt))
operation_time = np.zeros(len(dt))

for i, dti in enumerate(dt):
    start = time.time()
    Nt = int(round(time_span / dti)) + 1

    Ez, Hy, x= fdtd_1d_t(eps_rel, dx, dti, Nt, source_frequency, source_position, source_pulse_length)

    end = time.time()
    operation_time[i] = end - start


    # Interpolate Ez to match the reference grid
    Ez_interp = np.interp(x_0, x, Ez[-1])
    Ez_ref_interp = Ez_0 #np.interp(x, x_0, Ez_0[-1]) #reference grid do not need to be interpolated since it's fixed

    # Compute L2 norm of the error
    errors[i] = np.linalg.norm(Ez_interp - Ez_ref_interp) / np.linalg.norm(Ez_ref_interp)
    print(f"dt = {dti:.2e}, runtime = {end-start:.2f} s, L2 error = {errors[i]:.2e}")

To visualize the results in the following both, the operation time and the error defined by the norm are plotted.

In [ ]:
# Plot operation time vs. time step
plt.figure()
plt.plot(dt, operation_time, 'o-')
plt.xlabel('temporal step $dt$ [s]')
plt.ylabel('Operation time [s]')
# plt.xscale('log')  # Use logarithmic scale for better visualization
# plt.yscale('log')
plt.grid()
plt.title('Operation Time vs. Time Step')
# plt.savefig('comp_time_dt.png', bbox_inches="tight")
plt.show()

# Plot L2 error norm vs. time step
plt.figure()
plt.plot(dt, errors, 'o-')
plt.xlabel('temporal step $dt$ [s]')
plt.ylabel('L2 Norm of Error')
# plt.yscale('log')
# plt.xscale()
plt.grid()
plt.title('L2 Norm of Error vs. Time Step')
# plt.savefig('error_dt.png', bbox_inches="tight")
plt.show()

<center>
    
<img src="1d_convergence_dt_time.svg" alt="1d_convergence_dx_time" width="800" height="400"/>

Figure 3-3: The operation time for different discretized temporal size in 1D FDTD.

</center>

<center>
    
<img src="1d_convergence_dt_error.svg" alt="fdtd_1d_ez_postprocessing_snapshot" width="800" height="400"/>

Figure 3-4: $L_2$ norm errors for different discretized temporal size in 1D FDTD.

</center>

When having a look at the result for the convergence of $dt$ it is surely visible that a change of the temporal discretization by keeping the spatial one constant would not result in any reasonable physics. Since there is actually a problem during the computational calculation resulting in roughly zero operation time. But since the temporal and spatial component are coupled to each other this result was assumed. For a more reasonable change of the discretization steps one can have a look at the convergence test of the spatial step size, which already include the coupled change of the temporal step size.

#### 3.1.3. <a id='toc3_1_3_'></a>[Example: 1D](#toc0_)

In [ ]:
# simulation parameters
dx = 15e-9 # grid spacing [m]

Nx = int(round(x_span/dx)) + 1 # number of grid points

# %% create permittivity distribution and run simulation %%%%%%%%%%%%%%%%%%%%%%

# please add your code here
x = np.linspace(-x_span/2, x_span/2, Nx)

eps_rel = np.ones(Nx)
for i in range(Nx):
    if x[i] > x_interface:
        eps_rel[i] = n2**2
    else:
        eps_rel[i] = n1**2

for i, xi in enumerate(x):
    if abs(xi - source_position) < dx/2:
        source_position = i

Ez, Hy, x, t = fdtd_1d(eps_rel, dx, time_span, source_frequency, source_position, source_pulse_length)

# figures of magnetic field
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
indices = [50, 550, 800, 1300]

# Plot each index in a subplot
for ax, idx in zip(axs.ravel(), indices):
    ax.plot(x * 1e6, Z0*Hy[idx] * 1e6, color = 'orange')
    ax.vlines(x_interface * 1e6, ymin=-5, ymax=5, ls='--', color='k')
    ax.set_xlim(x[[0,-1]]*1e6)
    ax.set_xlabel('$x$ [µm]')
    ax.set_ylabel('$\\Re\\{H_y\\}$ [µV/m]')
    ax.set_title(f'Magnetic field at $ct = {c*t[idx]*1e6:.2f}$µm, i={idx}')
plt.tight_layout()
plt.show()

# figures of electric field
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
indices = [50, 550, 800, 1300]

# Plot each index in a subplot
for ax, idx in zip(axs.ravel(), indices):
    ax.plot(x * 1e6, Ez[idx] * 1e6)
    ax.vlines(x_interface * 1e6, ymin=-5, ymax=5, ls='--', color='k')
    ax.set_xlim(x[[0,-1]]*1e6)
    ax.set_xlabel('$x$ [µm]')
    ax.set_ylabel('$\\Re\\{E_z\\}$ [µV/m]')
    ax.set_title(f'Electric field at $ct = {c*t[idx]*1e6:.2f}$µm, i={idx}')
plt.tight_layout()
plt.show()



Add representative figures of each type of field with the following code. The specific propagation position is defined by observing the animation of the propagating fields.

In [ ]:
# figures of magnetic field
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
indices = [50, 550, 800, 1300]

# Plot each index in a subplot
for ax, idx in zip(axs.ravel(), indices):
    ax.plot(x * 1e6, Z0*Hy[idx] * 1e6, color = 'orange')
    ax.vlines(x_interface * 1e6, ymin=-5, ymax=5, ls='--', color='k')
    ax.set_xlim(x[[0,-1]]*1e6)
    ax.set_xlabel('$x$ [µm]')
    ax.set_ylabel('$\\Re\\{H_y\\}$ [µV/m]')
    ax.set_title(f'Magnetic field at $ct = {c*t[idx]*1e6:.2f}$µm, i={idx}')
plt.tight_layout()
plt.show()

# figures of electric field
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
indices = [50, 550, 800, 1300]

# Plot each index in a subplot
for ax, idx in zip(axs.ravel(), indices):
    ax.plot(x * 1e6, Ez[idx] * 1e6)
    ax.vlines(x_interface * 1e6, ymin=-5, ymax=5, ls='--', color='k')
    ax.set_xlim(x[[0,-1]]*1e6)
    ax.set_xlabel('$x$ [µm]')
    ax.set_ylabel('$\\Re\\{E_z\\}$ [µV/m]')
    ax.set_title(f'Electric field at $ct = {c*t[idx]*1e6:.2f}$µm, i={idx}')
plt.tight_layout()
plt.show()

This code sets up a 1D FDTD simulation to analyze electromagnetic wave propagation through a medium with two distinct permittivities separated by an interface. It begins by defining the spatial (*dx*) and temporal (*time_span*) resolutions and computes the number of grid points (*Nx+). The spatial grid (*x+) spans from *-x_span/2* to *x_span/2*, where *x_span* represents the total spatial extent of the simulation domain. The relative permittivity (*eps_rel*) is initialized to unity (vacuum) but modified to reflect different permittivities ($n_1^2$ and $n_2^2$) on either side of a specified interface (*x_interface*). The code identifies the grid index closest to the source position and adjusts it accordingly. Finally, it calls the fdtd_1d function to perform the simulation, yielding the electric field (*Ez*), magnetic field (*Hy*), spatial coordinates (*x*), and time steps (*t*).

<center>
    
<img src="fdtd_1d_ez_postprocessing_snapshot.svg" alt="fdtd_1d_ez_postprocessing_snapshot" width="1600" height="800"/>

Figure 3-5: $E_z$ field distribution at different time steps in 1D FDTD.

</center>

<center>
    
<img src="fdtd_1d_hx_postprocessing_snapshot.svg" alt="fdtd_1d_hx_postprocessing_snapshot" width="1600" height="800"/>

Figure 3-6: $H_x$ field distribution at different time steps in 1D FDTD.

</center>

For both fields the same positions are visualized as snapshots. In the first one at $ct = 0.38~\mu m$ the kick of the system is visible shortly before the pulse starts to propagate with both, forward and backward propagating direction.

The second image shows the fields at $ct = 4.13~\mu m$ where the further propagation of the pulses in homogenous media is visible.

The third image then shows the interaction of the pulse with the interface and the change of media at $ct=6.00~\mu m$. Which then result in a transmitted and a reflected part.

The fourth image then finally shows the last important behavior of the fields at $ct=9.75~\mu m$ in one circle of propagation: the backward propagating pulse hit the boundary and is perfectly reflected back.

Besides, when visualizing the results, we multiply the magnetic field components with impedance to keep the magnetic and electric field at the same order of magnitude.

Considering the fields at the boundary, we will discuss two things, i.e., the continuity at the boundary and the reflection and transmission effect at the boundary.

Since the material here is dielectrics and thus both the field components $E_z$ and $H_y$ are both continuous at the interface just as shown in the simulation results.

CConsidering the reflection and transmission, we assume that the light is normally incident. According to the Fresnel equations,

\begin{align}
t_s=\frac{2n_1}{n_1+n_2}=\frac{2}{3},\tag{}
r_s=\frac{n_1-n_2}{n_1+n_2}=-\frac{1}{3},\tag{}
\end{align}

the transmitted wave has larger amplitude than the reflected wave and reflected has a $\pi$ phase shit compared to the incident and transmitted wave. The simulation result is just consistent with the theory.

### 3.2. <a id='toc3_2_'></a>[Task 2 - 3D FDTD](#toc0_)

- Basic parameters - 3D

In [ ]:
# constants
c = 2.99792458e8 # speed of light [m/s]
mu0 = 4*np.pi*1e-7 # vacuum permeability [Vs/(Am)]
eps0 = 1/(mu0*c**2) # vacuum permittivity [As/(Vm)]
Z0 = np.sqrt(mu0/eps0) # vacuum impedance [Ohm]

# source parameters
freq = 500e12 # pulse [Hz]
tau = 1e-15 # pulse width [s]
source_width = 2 # width of Gaussian current dist. [grid points]

#### 3.2.1. <a id='toc3_2_1_'></a>[Implementation](#toc0_)

In [ ]:
def fdtd_3d(eps_rel, dr, time_span, freq, tau, jx, jy, jz,
            field_component, z_ind, output_step):
    '''Computes the temporal evolution of a pulsed spatially extended current source using the 3D FDTD method. Returns z-slices of the selected field at the given z-position every output_step time steps. The pulse is centered at a simulation time of 3*tau. All quantities have to be specified in SI units.

    Arguments
    ---------
        eps_rel: 3d-array
            Rel. permittivity distribution within the computational domain.
        dr: float
            Grid spacing (please ensure dr<=lambda/20).
        time_span: float
            Time span of simulation.
        freq: float
            Center frequency of the current source.
        tau: float
            Temporal width of Gaussian envelope of the source.
        jx, jy, jz: 3d-array
            Spatial density profile of the current source.
        field_component : str
            Field component which is stored (one of 'ex','ey','ez','hx','hy','hz').
        z_index: int
            Z-position of the field output.
        output_step: int
            Number of time steps between field outputs.

    Returns
    -------
        F: 3d-array
            Z-slices of the selected field component at the z-position specified by z_ind stored every output_step         time steps (time varies along the first axis).
        t: 1d-array
            Time of the field output.
    '''

    # basic parameters
    c = 2.99792458e8 # speed of light [m/s]
    mu0 = 4*np.pi*1e-7 # vacuum permeability [Vs/(Am)]
    eps0 = 1/(mu0*c**2) # vacuum permittivity [As/(Vm)]

    # time step
    dt = dr / (2 * c)
    Nt = int(round(time_span / dt)) + 1
    t = np.linspace(0, time_span, Nt)

    # construction of matrices
    ex = np.zeros((eps_rel.shape[0], eps_rel.shape[1], eps_rel.shape[2]))
    ey = np.zeros((eps_rel.shape[0], eps_rel.shape[1], eps_rel.shape[2]))
    ez = np.zeros((eps_rel.shape[0], eps_rel.shape[1], eps_rel.shape[2]))
    hx = np.zeros((eps_rel.shape[0], eps_rel.shape[1], eps_rel.shape[2]))
    hy = np.zeros((eps_rel.shape[0], eps_rel.shape[1], eps_rel.shape[2]))
    hz = np.zeros((eps_rel.shape[0], eps_rel.shape[1], eps_rel.shape[2]))

    F1 = []
    F2 = []
    Ex = []
    Ey = []
    Ez = []
    Hx = []
    Hy = []
    Hz = []

    # Main loop
    for n in range(0, Nt):
        # Add perfect electric conductor boundary conditions
        ex[:, 0, :] = 0
        ex[:, -1, :] = 0
        ex[:, :, 0] = 0
        ex[:, :, -1] = 0
        ey[0, :, :] = 0
        ey[-1, :, :] = 0
        ey[:, :, 0] = 0
        ey[:, :, -1] = 0
        ez[0, :, :] = 0
        ez[-1, :, :] = 0
        ez[:, 0, :] = 0
        ez[:, -1, :] = 0
        hx[0, :, :] = 0
        hx[-1, :, :] = 0
        hy[:, 0, :] = 0
        hy[:, -1, :] = 0
        hz[:, :, 0] = 0
        hz[:, :, -1] = 0


        # Update electric fields
        ex = ex + dt / (eps0 * eps_rel) * ((hz - np.roll(hz, 1, axis=1)) - (hy - np.roll(hy, 1, axis=2))) / dr - jx * np.cos(2 * np.pi * freq * (n + 0.5) * dt) * np.exp(-(((n + 0.5) * dt - 3 * tau) / tau) ** 2) * dt / (eps0  * eps_rel)

        ey = ey + dt / (eps0  * eps_rel) * ((hx - np.roll(hx, 1, axis=2)) - (hz - np.roll(hz, 1, axis=0))) / dr - jy * np.cos(2 * np.pi * freq * (n + 0.5) * dt) * np.exp(-(((n + 0.5) * dt - 3 * tau) / tau) ** 2) * dt / (eps0  * eps_rel)

        ez = ez + dt / (eps0) * ((hy - np.roll(hy, 1, axis=0)) - (hx - np.roll(hx, 1, axis=1))) / dr - jz * np.cos(2 * np.pi * freq * (n + 0.5) * dt) * np.exp(-(((n + 0.5) * dt - 3 * tau) / tau) ** 2) * dt / (eps0)

        # Update magnetic fields
        hx = hx - dt / mu0 * ((ey - np.roll(ey, -1, axis=2)) - (ez - np.roll(ez, -1, axis=1))) / dr

        hy = hy - dt / mu0 * ((ez - np.roll(ez, -1, axis=0)) - (ex - np.roll(ex, -1, axis=2))) / dr

        hz = hz - dt / mu0 * ((ex - np.roll(ex, -1, axis=1)) - (ey - np.roll(ey, -1, axis=0))) / dr

        # Save the field components for a specific z-plane index `z_ind`
        # F1.append(hx[:, :, z_ind])
        # F2.append(ez[:, :, z_ind])

        # Save the field components at a specific time
        Ex.append(ex)
        Ey.append(ey)
        Ez.append(ez)
        Hx.append(hx)
        Hy.append(hy)
        Hz.append(hz)


    # F1 = np.array(F1)
    # F2 = np.array(F2)
    Ex = np.array(Ex)
    Ey = np.array(Ey)
    Ez = np.array(Ez)
    Hx = np.array(Hx)
    Hy = np.array(Hy)
    Hz = np.array(Hz)

    # Postprocessing - interpolation of output
    Ex = 0.5 * (Ex + np.roll(Ex, 1, axis=1))

    Ey = 0.5 * (Ey + np.roll(Ey, 1, axis=2))

    Ez = 0.5 * (Ez + np.roll(Ez, 1, axis=3))

    Hx = 0.125 * (Hx + np.roll(Hx, 1, axis=2) + Hx + np.roll(Hx, 1, axis=3) + np.roll(np.roll(Hx, 1, axis=2), 1, axis=3) + np.roll((Hx + np.roll(Hx, 1, axis=2) + Hx + np.roll(Hx, 1, axis=3) + np.roll(np.roll(Hx, 1, axis=2), 1, axis=3)), 1, axis=0))

    Hy = 0.125 * (Hy + np.roll(Hy, 1, axis=1) + Hy + np.roll(Hy, 1, axis=3) + np.roll(np.roll(Hy, 1, axis=1), 1, axis=3) + np.roll((Hy + np.roll(Hy, 1, axis=1) + Hy + np.roll(Hy, 1, axis=3) + np.roll(np.roll(Hy, 1, axis=1), 1, axis=3)), 1, axis=0))

    Hz = 0.125 * (Hz + np.roll(Hz, 1, axis=1) + Hz + np.roll(Hz, 1, axis=2) + np.roll(np.roll(Hz, 1, axis=1), 1, axis=2) + np.roll((Hz + np.roll(Hz, 1, axis=1) + Hz + np.roll(Hz, 1, axis=2) + np.roll(np.roll(Hz, 1, axis=1), 1, axis=2)), 1, axis=0))

    F1 = np.zeros((len(t), eps_rel.shape[0], eps_rel.shape[1]))
    F2 = np.zeros((len(t), eps_rel.shape[0], eps_rel.shape[1]))
    if field_component == 'hx' or 'ez':

            for n in range(0, len(t)):
                F1[n, :, :] = Hx[n, :, :, z_ind]
                F2[n, :, :] = Ez[n, :, :, z_ind]

            F1 = F1[::output_step, :, :]
            F2 = F2[::output_step, :, :]

    t = t[::output_step]

    return F1, F2, t

The function fdtd_3d implements a 3D Finite-Difference Time-Domain (FDTD) simulation to model the time evolution of electromagnetic fields in a three-dimensional space. The function is designed to simulate how a pulsed, spatially extended current source interacts with a medium characterized by a given relative permittivity distribution. It returns slices of the electromagnetic field components at a specific z-coordinate, allowing for detailed analysis of field behavior over time.

The fdtd_3d function begins by defining essential physical constants such as the speed of light (c), vacuum permeability (μ₀), and vacuum permittivity (ε₀). These constants are used to calculate the time step (dt), ensuring numerical stability by satisfying the Courant stability condition, which is critical for accurate simulation of electromagnetic wave propagation. The number of time steps (Nt) is derived from the total simulation duration (time_span), and a time array (t) is created for tracking the simulation progress. Zero-filled matrices for electric (ex, ey, ez) and magnetic (hx, hy, hz) fields are initialized to store the field values at each grid point in the computational domain. The function enforces perfect electric conductor (PEC) boundary conditions by setting the field components to zero at the edges of the computational domain, simulating reflective boundaries that prevent fields from escaping the simulation area.

The main simulation loop iterates over each time step to update the electric and magnetic fields using Maxwell's equations. The electric fields are updated based on the curl of the magnetic fields and current source terms (jx, jy, jz), which are modulated by a Gaussian envelope and a sinusoidal function to represent the pulse. The magnetic fields are updated similarly using the curl of the electric fields, with spatial derivatives implemented through the np.roll function, which shifts field values along different axes. The function collects the specified field components (ex, ey, ez, hx, hy, hz) at a given z-plane index (z_ind) and stores them in arrays, allowing for examination of field evolution at a specific cross-section. Post-processing involves interpolating the fields to average values over adjacent grid points, smoothing the data to better represent continuous fields. The function returns the selected field slices (F1 and F2) and the corresponding time values (t), saving data at intervals specified by output_step to focus on key time points and manage data efficiently.

**1. Convergence test for dx**

**2. Convergence test for dt**

#### 3.2.2. <a id='toc3_2_2_'></a>[Example](#toc0_)
This executes a 3D Finite-Difference Time-Domain (FDTD) simulation to analyze the electromagnetic field distribution in a 3D space. The simulation parameters define the computational domain, the material properties, the current source characteristics, and the output requirements. The goal is to understand the interaction of electromagnetic waves with a medium characterized by a relative permittivity distribution and a current source.

In [ ]:
# simulation parameters
Nx = 199 # number of grid points in x-direction
Ny = 201 # number of grid points in y-direction
Nz = 5   # number of grid points in z-direction
dr = 30e-9 # grid spacing in [m]
time_span = 10e-15 # duration of simulation [s]

# x coordinates
x = np.arange(-int(np.ceil((Nx-1)/2)), int(np.floor((Nx-1)/2)) + 1)*dr
# y coordinates
y = np.arange(-int(np.ceil((Ny-1)/2)), int(np.floor((Ny-1)/2)) + 1)*dr

# grid midpoints
midx = int(np.ceil((Nx-1)/2))
midy = int(np.ceil((Ny-1)/2))
midz = int(np.ceil((Nz-1)/2))

# %% create relative permittivity distribution %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# eps_rel = ...

eps_rel = np.ones((Nx, Ny, Nz))

# %% current distributions %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# jx = jy = np.zeros(...)
# jz : Gaussion distribution in the xy-plane with a width of 2 grid points,
# constant along z

# jx = ...
# jy = ...
# jz = ...

jx = np.zeros((Nx, Ny, Nz))
jy = np.zeros((Nx, Ny, Nz))
jz = np.zeros((Nx, Ny, Nz))
for i, xi in enumerate(x):
    for j, yj in enumerate(y):
        jz[i, j, :] = np.exp(-((xi)**2 + (yj)**2)/(source_width**2))

# output parameters
z_ind = midz # z-index of field output
output_step = 4 # time steps between field output

field_component = 'hx'
hx, ez, t = fdtd_3d(eps_rel, dr, time_span, freq, tau, jx, jy, jz, field_component, z_ind, output_step)

Add representative figures of each type of field with the following code. The specific propagation position is defined by observing the animation of the propagating fields.

In [ ]:
# create representative figures of the results 

def figures(field, t, x, y, time_steps, rel_color_range=1/3, field_type='', cb_label=''):
    ct = c * t  # time in seconds converted to distance

    num_plots = len(time_steps)
    nrows = int(np.ceil(num_plots / 2))
    ncols = 2 if num_plots > 1 else 1

    fig, axes = plt.subplots(nrows, ncols, figsize=(10, 5 * nrows))
    axes = axes.flatten() if num_plots > 1 else [axes]

    F = field
    color_range = rel_color_range * np.max(np.abs(field))
    phw = 0.5 * (x[1] - x[0])  # pixel half-width
    extent = ((x[0] - phw) * 1e6, (x[-1] + phw) * 1e6, 
              (y[-1] + phw) * 1e6, (y[0] - phw) * 1e6)
    
    img = None
    for idx, ax in enumerate(axes):
        if idx < num_plots:
            time_step = time_steps[idx]
            img = ax.imshow(F[time_step,:,:].real.T, vmin=-color_range, vmax=color_range, extent=extent)
            ax.invert_yaxis()
            ax.set_title(f'$ct = {c*t[time_step]*1e6:.2f}$ µm')
            ax.set_xlabel('x position [µm]')
            ax.set_ylabel('y position [µm]')
            
        else:
            fig.delaxes(ax)

    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.17, 0.05, 0.7])
    cb = fig.colorbar(img, cax=cbar_ax)
    cb.set_label(cb_label)
    fig.suptitle(f'{field_type} field distribution at different time steps')
    plt.savefig(f'3d_figures_{field_type}.png', bbox_inches="tight")
    plt.show()

indices = [10,25,40,50]
figures(ez*1e6, t, x, y, indices, rel_color_range=1/3, field_type='Electric', cb_label='$\\Re\\{E_z\\}$ [µV/m]')

figures(hx*1e6, t, x, y, indices, rel_color_range=1/3, field_type='Magnetic', cb_label='$\\Re\\{Z_0H_x\\}$ [µV/m]' )

<center>
    
<img src="fdtd_3d_ez_postprocessing_snapshot.svg" alt="fdtd_3d_ez_postprocessing_snapshot" width="1600" height="800"/>

Figure 3-: $E_z$ field distribution at different time steps in 3D FDTD.

</center>

<center>
    
<img src="fdtd_3d_hx_postprocessing_snapshot.svg" alt="fdtd_3d_hx_postprocessing_snapshot" width="1600" height="800"/>

Figure 3-: $H_x$ field distribution at different time steps in 3D FDTD.

</center>

## 4. <a id='toc4_'></a>[Conclusion](#toc0_)

## 5. <a id='toc5_'></a>[References](#toc0_)

[1]. Thomas Pertsch (2024): Chapter 6 - Finite-Difference Time-Domain (FDTD) Method.
   In Thomas Pertsch: Computational Photonics: Abbe School of Photonics, FSU Jena, pp. 75-103.

[2]. Thomas Pertsch (2024): Computational Photonics: Seminar 6, June 14, 2024: Finite-Difference Time-Domain Method (FDTD).